# DACON - 태양광 발전량 예측 competition
***

## 1. Package Import

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import GradientBoostingRegressor
import os
from lightgbm import LGBMRegressor
import datetime

***

## 2. Load Data

In [3]:
train = pd.read_csv('/Users/kisehyun/Competition/데이콘_태양광 발전량 예측/train/train.csv')

for i in range(81) :
    globals() [f'test_{i}'] = pd.read_csv(f'/Users/kisehyun/Competition/데이콘_태양광 발전량 예측/test/{i}.csv')
    globals() [f'test_{i}']['sep_day'] = i
    
### 각 데이터를 하나의 데이터로 통합
test = pd.DataFrame()
idx_list = []
for i in range(81):
    test = pd.concat([test, globals()[f'test_{i}']], axis = 0, ignore_index = True)
    
submission = pd.read_csv('sample_submission.csv')

***

## 3. Setting


### Loss Function Definition


In [4]:
def pb_loss(true, pred, q) :
    
    L_list = []
    
    for i in range(len(true)) :
        
        if true.iloc[i] >= pred[i] :
            L = true.iloc[i] - pred[i]
            L *= q
            
        else :
            L1 = pred[i] - true.iloc[i]
            L2 = 1 - q
            L = L1 * L2
            
        L_list.append(L)
        
    return np.mean(L_list)

In [5]:
#train['N/H'] = train.DNI / (train.DHI + train.DNI)
#test['N/H'] = test.DNI / (test.DHI + test.DNI)

#train.fillna(0, inplace = True)
#test.fillna(0, inplace = True)

### custom definition to make dataset
- after 1 day

- after 2 days

In [6]:
def make_data() :
    ### 1일 후 데이터
    train_after_1 = train.loc[train.Day <= 1093]
    train_after_1['TARGET1'] = list(train.loc[train.Day >= 1].TARGET)
    
    ### 2일 후 데이터
    train_after_2 = train.loc[train.Day <= 1092]
    train_after_2['TARGET2'] = list(train.loc[train.Day >= 2].TARGET)
    
    ### after 1 day data 
    a_1day = test.loc[test.Day != 6].drop(['Day', 'Minute','sep_day','DHI', 'RH', 'WS'], axis = 1)
    
    ### after 2 day data 
    a_2day = test.query('Day not in [5, 6]').drop(['Day', 'Minute','sep_day','DHI', 'RH', 'WS'], axis = 1)
    
    ### after 1 day train data
    X_1 = train_after_1.drop(['Day','Minute', 'TARGET1','DHI', 'RH', 'WS'], axis = 1)
    y_1 = train_after_1.TARGET1
    
    ### after 2 days train data
    X_2 = train_after_2.drop(['Day','Minute', 'TARGET2','DHI', 'RH', 'WS'], axis = 1)
    y_2 = train_after_2.TARGET2
    
    ### actual values
    real_1 = test.loc[test.Day != 0].TARGET
    real_2 = test.query('Day not in [0, 1]').TARGET
    
    target = test.loc[test.Day == 6].drop(['Day', 'sep_day', 'Minute','DHI', 'RH', 'WS'], axis = 1)
    
    return a_1day, a_2day, X_1, y_1, X_2, y_2, real_1, real_2, target

In [7]:
#a_1day, a_2day, X_1, y_1, X_2, y_2, real_1, real_2, target = make_data()

## 4. Modeling

- 4-1. Baseline Modeling

In [8]:
"""t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'###### {qt + 1}번째 학습 및 검증 시작 ######')
    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 10000, learning_rate=0.027, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)
    lgbm.fit(X_1, y_1, eval_set = [(a_1day, real_1)], early_stopping_rounds = 1000, verbose = 0)
    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])
    print(f'best loss는 {best_loss}\n')
    t_loss.append(best_loss)
    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]
    submission.loc[submission.id.str.contains('y7'), submission.columns[1+ qt]] = lgbm_pred
lgbm_submission = submission.copy()
print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')"""

"t_loss = []\nqt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]\n\nfor qt in range(9) :\n    print(f'###### {qt + 1}번째 학습 및 검증 시작 ######')\n    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 10000, learning_rate=0.027, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)\n    lgbm.fit(X_1, y_1, eval_set = [(a_1day, real_1)], early_stopping_rounds = 1000, verbose = 0)\n    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])\n    print(f'best loss는 {best_loss}\n')\n    t_loss.append(best_loss)\n    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]\n    submission.loc[submission.id.str.contains('y7'), submission.columns[1+ qt]] = lgbm_pred\nlgbm_submission = submission.copy()\nprint(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')\nprint(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')"

In [9]:
'''
t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'###### {qt + 1}번째 학습 및 검증 시작 ######')
    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 10000, learning_rate=0.027, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)
    lgbm.fit(X_2, y_2, eval_set = [(a_2day, real_2)], early_stopping_rounds = 1000, verbose = 0)
    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])
    print(f'best loss는 {best_loss}\n')
    t_loss.append(best_loss)
    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]
    lgbm_submission.loc[lgbm_submission.id.str.contains('y8'), lgbm_submission.columns[1+ qt]] = lgbm_pred
lgbm_submission = submission.copy()
print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')'''

"\nt_loss = []\nqt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]\n\nfor qt in range(9) :\n    print(f'###### {qt + 1}번째 학습 및 검증 시작 ######')\n    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 10000, learning_rate=0.027, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)\n    lgbm.fit(X_2, y_2, eval_set = [(a_2day, real_2)], early_stopping_rounds = 1000, verbose = 0)\n    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])\n    print(f'best loss는 {best_loss}\n')\n    t_loss.append(best_loss)\n    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]\n    lgbm_submission.loc[lgbm_submission.id.str.contains('y8'), lgbm_submission.columns[1+ qt]] = lgbm_pred\nlgbm_submission = submission.copy()\nprint(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')\nprint(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')"

In [10]:
#lgbm_submission.describe()

In [11]:
#lgbm_submission.to_csv('lgbm_baseline.csv', index = False)

#### it's score was bad. so, i made some features to get better score.

1. 일조시간

In [12]:
tr_times = []
for day in train.Day.unique() :
    d = train.loc[train.Day == day]
    try :
        start = d.loc[d.DHI > 0].index[0]
        end = d.loc[d.DHI > 0].index[-1]
        gap = (end - start) / 2
    except :
        gap = 0

    tr_times.append(gap)

In [13]:
test['sep'] = test.index // 48 # test 데이터의 경우 시계열 고려가 안되었기 때문에 48(하루 시간)으로 나눈 몫으로 day 구분자 생성


te_times = []
for day in test.sep.unique() :
    d = test.loc[test.sep == day]
    try :
        start = d.loc[d.DHI > 0].index[0]
        end = d.loc[d.DHI > 0].index[-1]
        gap = (end - start) / 2
    except :
        gap = 0

    te_times.append(gap)

In [14]:
train_sun_time = pd.DataFrame({'Day' : train.Day.unique(), 'sun_time' : tr_times})
test_sun_time = pd.DataFrame({'sep' : test.sep.unique(), 'sun_time' : te_times})

In [15]:
train = pd.merge(train, train_sun_time, how = 'left', on = 'Day')
test = pd.merge(test, test_sun_time, how = 'left', on = 'sep')#.drop('sep', axis = 1)

2. month

In [16]:
train['new_day'] = train.Day + 1
train.new_day %= 365
train

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,sun_time,new_day
0,0,0,0,0,0,1.5,69.08,-12,0.0,8.5,1
1,0,0,30,0,0,1.5,69.06,-12,0.0,8.5,1
2,0,1,0,0,0,1.6,71.78,-12,0.0,8.5,1
3,0,1,30,0,0,1.6,71.75,-12,0.0,8.5,1
4,0,2,0,0,0,1.6,75.20,-12,0.0,8.5,1
...,...,...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0,8.5,0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0,8.5,0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0,8.5,0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0,8.5,0


In [17]:
'''def get_degree(x) :
    if x == 0 :
        return 29
    elif x <= 5 : 
        return 32.92
    elif x <= 20 :
        return 36.83
    elif x <= 35 :
        return 40.75
    elif x <= 53 :
        return 44.67
    elif x <= 64 :
        return 46.58
    elif x <= 79 :
        return 52.5
    elif x <= 94 :
        return 56.42
    elif x <= 110 :
        return 60.33
    elif x <= 125 :
        return 64.25
    elif x <= 141 :
        return 68.16
    elif x <= 156 :
        return 72.01
    elif x <= 172 :
        return 76
    elif x <= 188 :
        return 72.1
    elif x <= 204 :
        return 68.16
    elif x <= 219 :
        return 64.25
    elif x <= 235 :
        return 60.33
    elif x <= 250 :
        return 56.42
    elif x <= 266 :
        return 52.5
    elif x <= 281 :
        return 48.58
    elif x <= 296 :
        return 44.67
    elif x <= 311 :
        return 40.75
    elif x <= 326 :
        return 36.83
    elif x <= 341 :
        return 32.92
    else :
        return 29
        '''

'def get_degree(x) :\n    if x == 0 :\n        return 29\n    elif x <= 5 : \n        return 32.92\n    elif x <= 20 :\n        return 36.83\n    elif x <= 35 :\n        return 40.75\n    elif x <= 53 :\n        return 44.67\n    elif x <= 64 :\n        return 46.58\n    elif x <= 79 :\n        return 52.5\n    elif x <= 94 :\n        return 56.42\n    elif x <= 110 :\n        return 60.33\n    elif x <= 125 :\n        return 64.25\n    elif x <= 141 :\n        return 68.16\n    elif x <= 156 :\n        return 72.01\n    elif x <= 172 :\n        return 76\n    elif x <= 188 :\n        return 72.1\n    elif x <= 204 :\n        return 68.16\n    elif x <= 219 :\n        return 64.25\n    elif x <= 235 :\n        return 60.33\n    elif x <= 250 :\n        return 56.42\n    elif x <= 266 :\n        return 52.5\n    elif x <= 281 :\n        return 48.58\n    elif x <= 296 :\n        return 44.67\n    elif x <= 311 :\n        return 40.75\n    elif x <= 326 :\n        return 36.83\n    elif 

In [18]:
#train['degree'] = train.new_day.apply(get_degree)

In [19]:
#d_t_mean = train.groupby('degree')['T'].mean()
#d_t_mean

In [20]:
#te_t_mean = test.groupby('sep_day')['T'].mean().reset_index()
#te_t_mean

In [21]:
#infer_degree = []
#for r in range(te_t_mean.shape[0]) :
#    infer_degree.append(d_t_mean.index[np.argmin(abs(d_t_mean - te_t_mean['T'][r]))])
#te_t_mean['degree'] = infer_degree

In [22]:
#test = pd.merge(test, te_t_mean[['sep_day', 'degree']], how = 'left', on = 'sep_day')

2. GHI

In [23]:

def get_d(x) :
    # 평균 기온 기준으로 계절별 평균 남중 고도 적용
    if x < 6.8 :
        return 29.5
    elif 6.8 <= x <= 20 :
        return 53
    else :
        return 76.5

In [24]:
tr_degree = train.groupby('Day')['T'].mean().reset_index(name = 'degree')
tr_degree.index.name = None
tr_degree.head()

,Day,degree
0,0,-7.979167
1,1,-6.312500
2,2,-6.479167
3,3,-5.687500
4,4,0.854167


In [25]:
te_degree = test.groupby('sep')['T'].mean().reset_index(name = 'degree')
te_degree.index.name = None
te_degree.head()

,sep,degree
0,0,4.652083
1,1,5.450000
2,2,3.414583
3,3,0.512500
4,4,-2.258333


In [26]:
tr_degree['degree'] = tr_degree.degree.apply(get_d)
te_degree['degree'] = te_degree.degree.apply(get_d)

In [27]:
train = pd.merge(train, tr_degree, how = 'left', on = 'Day')
test = pd.merge(test, te_degree, how = 'left', on = 'sep')

In [28]:
train['GHI'] = train.DNI * np.cos(train.degree) + train.DHI
test['GHI'] = test.DNI * np.cos(test.degree) + test.DHI

In [29]:
train.drop(['degree', 'new_day'], axis = 1, inplace = True)
test.drop(['degree', 'sep'], axis = 1, inplace = True)

I tried Log transformation but it was not good.

In [30]:
#train['N/H'] = np.log1p(train['N/H'])
#test['N/H'] = np.log1p(test['N/H'])

In [31]:
#train.DNI = np.log1p(train.DNI)
#test.DNI = np.log1p(test.DNI)

In [32]:
#train.WS = np.log1p(train.WS)
#test.WS = np.log1p(test.WS)

In [33]:
a_1day, a_2day, X_1, y_1, X_2, y_2, real_1, real_2, target = make_data()

- 4-2. 2nd Modeling

#### LGBMRegressor

In [35]:
t_loss = []
qt_list = [.08, .18, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'##### {qt + 1}번째 학습 및 검증 시작 #####')
    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 1000, learning_rate=0.03, metric = 'quantile', alpha = qt_list[qt], max_depth = 5)
    lgbm.fit(X_1, y_1, eval_set = [(a_1day, real_1)], early_stopping_rounds = 300, verbose = 500)
    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])
    print(f'best loss는 {best_loss}\n')
    t_loss.append(best_loss)
    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]
    submission.loc[submission.id.str.contains('y7'), submission.columns[1+ qt]] = lgbm_pred
lgbm_submission = submission.copy()

print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')

##### 1번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 300 rounds
[500]	valid_0's quantile: 1.19175
Early stopping, best iteration is:
[372]	valid_0's quantile: 1.18695
best loss는 1.186949147725634

##### 2번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[160]	valid_0's quantile: 2.1548
best loss는 2.154796096710185

##### 3번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 300 rounds
[500]	valid_0's quantile: 2.69022
Early stopping, best iteration is:
[267]	valid_0's quantile: 2.68673
best loss

In [36]:
lgbm_submission.loc[lgbm_submission.id.str.contains('y7')].describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,3888.000000,3888.000000,3888.000000,3888.000000,3888.000000,3888.000000,3.888000e+03,3.888000e+03,3.888000e+03
mean,6.531792,11.144956,15.067863,17.510145,19.928488,21.402269,2.253716e+01,2.359581e+01,2.475667e+01
std,10.175262,15.613434,20.452535,23.385614,26.179160,27.908492,2.916489e+01,3.026514e+01,3.118054e+01
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.726929e-10,0.000000e+00,5.552333e-12
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000084,2.726929e-10,4.921867e-11,5.552333e-12
50%,0.148399,0.573518,1.093955,1.394947,1.862336,1.847237,2.317348e+00,2.732368e+00,4.196523e+00
75%,11.208107,19.910887,28.430858,33.786190,39.030880,42.875148,4.519145e+01,4.730744e+01,4.960946e+01
max,56.160770,62.242857,78.336744,86.990682,90.982292,93.659420,9.607490e+01,9.693431e+01,9.797086e+01


In [37]:
t_loss = []
qt_list = [.08, .19, .28, .37, .48, .59, .68, .8, .89]

for qt in range(9) :
    print(f'##### {qt + 1}번째 학습 및 검증 시작 #####')
    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 1000, learning_rate=0.03, metric = 'quantile', alpha = qt_list[qt], max_depth = 5)
    lgbm.fit(X_2, y_2, eval_set = [(a_2day, real_2)], early_stopping_rounds = 1000, verbose = 0)
    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])
    print(f'best loss는 {best_loss}\n')
    t_loss.append(best_loss)
    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]
    submission.loc[submission.id.str.contains('y8'), submission.columns[1+ qt]] = lgbm_pred
lgbm_submission = submission.copy()

print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')

##### 1번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
best loss는 1.2271124596085599

##### 2번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
best loss는 2.355149084054306

##### 3번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
best loss는 2.859367239228032

##### 4번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
best loss는 3.0270726359314177

##### 5번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
best loss는 2.9252067041505927

##### 6번째 학습 및 검증 시작 #####
[LightGBM] [Warning] Accuracy may b

In [38]:
lgbm_submission.loc[lgbm_submission.id.str.contains('y8')].describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,3888.000000,3888.000000,3888.000000,3888.000000,3888.000000,3.888000e+03,3888.000000,3.888000e+03,3.888000e+03
mean,4.449709,11.100146,15.055729,17.474134,20.082875,2.171820e+01,22.683492,2.373009e+01,2.468072e+01
std,6.230229,15.228535,20.226367,22.913317,26.203249,2.823455e+01,29.404666,3.036631e+01,3.112196e+01
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000002,0.000000e+00,0.000000e+00
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.321795e-07,0.000002,2.487574e-12,4.264613e-12
50%,0.109289,0.493746,1.032380,1.425910,1.779346,1.727857e+00,2.123406,2.786944e+00,3.642745e+00
75%,8.437596,21.351151,29.697734,35.119088,40.830432,4.368921e+01,45.064641,4.716635e+01,4.949652e+01
max,21.620965,71.308691,86.964537,81.700725,90.906423,9.768722e+01,96.634367,9.648956e+01,9.819383e+01


#### GradientBoostingRegressor

In [39]:
t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'##### {qt + 1}번째 학습 및 검증 시작 #####')
    gbm = GradientBoostingRegressor(random_state = 1231, loss = 'quantile', alpha = qt_list[qt], max_depth = 4)
    gbm.fit(X_1, y_1)
    gbm_pred = gbm.predict(a_1day)
    loss = pb_loss(real_1, gbm_pred, qt_list[qt])
    print(f'loss는 {loss}\n')
    t_loss.append(loss)
    gbm_pred = [0 if x < 0 else x for x in gbm.predict(target)]
    submission.loc[submission.id.str.contains('y7'), submission.columns[1+ qt]] = gbm_pred
gbm_submission = submission.copy()
print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')

##### 1번째 학습 및 검증 시작 #####
loss는 1.3072296885912247

##### 2번째 학습 및 검증 시작 #####
loss는 2.2333799123786293

##### 3번째 학습 및 검증 시작 #####
loss는 2.702664971695213

##### 4번째 학습 및 검증 시작 #####
loss는 2.883545304034839

##### 5번째 학습 및 검증 시작 #####
loss는 2.802381552558668

##### 6번째 학습 및 검증 시작 #####
loss는 2.504732699050954

##### 7번째 학습 및 검증 시작 #####
loss는 1.9962801406427317

##### 8번째 학습 및 검증 시작 #####
loss는 1.5013040249081226

##### 9번째 학습 및 검증 시작 #####
loss는 0.8811780266368845

전체 loss 평균은 2.0903
전체 loss 편차는 0.67778


In [40]:
t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'##### {qt + 1}번째 학습 및 검증 시작 #####')
    gbm = GradientBoostingRegressor(random_state = 1231, loss = 'quantile', alpha = qt_list[qt], max_depth = 5)
    gbm.fit(X_2, y_2)
    gbm_pred = gbm.predict(a_2day)
    loss = pb_loss(real_2, gbm_pred, qt_list[qt])
    print(f'loss는 {loss}\n')
    t_loss.append(loss)
    gbm_pred = [0 if x < 0 else x for x in gbm.predict(target)]
    submission.loc[submission.id.str.contains('y8'), submission.columns[1+ qt]] = gbm_pred
gbm_submission = submission.copy()
print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')

##### 1번째 학습 및 검증 시작 #####
loss는 1.3635291894542325

##### 2번째 학습 및 검증 시작 #####
loss는 2.358905086558488

##### 3번째 학습 및 검증 시작 #####
loss는 2.877274204786395

##### 4번째 학습 및 검증 시작 #####
loss는 3.0457535401428606

##### 5번째 학습 및 검증 시작 #####
loss는 2.9556967598189465

##### 6번째 학습 및 검증 시작 #####
loss는 2.586157941770812

##### 7번째 학습 및 검증 시작 #####
loss는 2.0813157081325064

##### 8번째 학습 및 검증 시작 #####
loss는 1.495374350810308

##### 9번째 학습 및 검증 시작 #####
loss는 0.8681419576108316

전체 loss 평균은 2.18135
전체 loss 편차는 0.7379


In [41]:
gbm_submission.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,6.433660,11.529091,14.980445,17.457919,19.889657,21.452330,22.648594,23.726207,25.025020
std,9.612667,16.080228,20.220858,23.136083,25.982479,27.921090,29.244698,30.211689,31.054788
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000204,0.000631,0.001084,0.002305
50%,0.121338,0.515678,1.080063,1.341532,1.736173,1.657615,2.109484,3.301418,5.470189
75%,11.394673,20.959608,28.506379,34.257715,40.077193,42.786107,45.267707,47.367673,49.872216
max,51.051285,70.835505,87.706069,88.768272,92.141226,97.082909,97.829392,99.366117,99.190458


## 5. Submission

In [47]:
submission.iloc[:, 1:] = lgbm_submission.iloc[:, 1:] * .7 + gbm_submission.iloc[:, 1:] * .3

In [48]:
zero_t = submission.loc[submission.id.str.contains('_0h|_1h|_2h|_3h|_4h|_21h|_22h|_23h')]
zero_t.iloc[:, 1:] = 0

In [49]:
submission.loc[zero_t.index] = zero_t

In [50]:
submission.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,5.773129,11.243243,15.035892,17.479719,19.968224,21.525454,22.618383,23.674899,24.794678
std,8.761572,15.605122,20.289908,23.134194,26.116274,28.014594,29.263203,30.275249,31.116232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.115589,0.518472,1.038338,1.368034,1.796835,1.707558,2.155901,2.999686,4.441578
75%,10.156388,20.799572,28.897203,34.413612,40.123064,43.139381,45.347562,47.184410,49.634646
max,51.268598,71.166735,87.186997,86.286345,91.129887,97.505927,96.133914,96.795309,98.058449


In [51]:
save_time = datetime.datetime.strftime(datetime.datetime.today(),'%Y%m%d_%H%M%S')

submission.to_csv(f'sun_{save_time}.csv', index=False)